In [7]:
# !wget https://huggingface.co/datasets/TokenBender/Tamil_chat_dataset/resolve/main/combined_data_no_duplicates_reloaded.jsonl
# !wget https://huggingface.co/datasets/abhinand/tamil-alpaca/resolve/main/data/train-00000-of-00001-e96a6e4e7f400cce.parquet

In [2]:
roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

system = [
    'always reply in tamil',
    'sentiasa respond dalam bahasa tamil',
    'sentiasa jawab in tamil',
    'awak adalah chatbot yang respond dalam bahasa tamil',
    'you are a chatbot that always respond in tamil',
    'u always reply in tamil',
    'awak sentiasa reply in tamil',
    'you are a chatbot that always reply in tamil',
    'you are an assistant that always reply in tamil',
    'awak adalah chatbot yang sentiasa respond dalam tamil',
    'awak adalah pembantu AI yang reply in tamil',
    'நீங்கள் எப்போதும் தமிழில் பதிலளிக்கும் உதவியாளர்',
]

In [3]:
import json
import random

In [4]:
instructions = []

with open('combined_data_no_duplicates_reloaded.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        translated = l['translated_text'].split('###')[1:]
        if len(l['conversations']) != len(translated):
            continue
            
        translated = [': '.join(t.split(': ')[1:]) for t in translated]
        r = []
        for i in range(len(l['conversations'])):
            
            try:
                splitted = translated[i].split()

                if (len(set(splitted)) / len(splitted)) < 0.7:
                    break
            except:
                break
            
            l['conversations'][i]['content_tamil'] = translated[i]
            
            if i % 2 == 0:
                k = random.choice(['value', 'content_tamil'])
                role = '<manusia>'
            else:
                k = 'content_tamil'
                role = '<bot>'
            
            r.append(f"{role}: {l['conversations'][i][k]}")
        
        if len(r) % 2 != 0:
            r = r[:-1]
        
        if not len(r):
            continue
        
        r[0] = r[0].replace('<manusia>:', f'<manusia>:{random.choice(system)}\n\n')
        instructions.append('\n'.join(r))
            
len(instructions)     

358

In [8]:
from tqdm import tqdm
import pandas as pd
df = pd.read_parquet('train-00000-of-00001-e96a6e4e7f400cce.parquet')
    
for i in tqdm(range(len(df))):
    l = df['instruction'].iloc[i]
    r = df['output'].iloc[i]
    
    if len(l) < 3 or len(r) < 3:
        continue
        
    results =[
        f"<manusia>: {l}",
        f"<bot>: {r}"
    ]
    results[0] = results[0].replace('<manusia>:', f'<manusia>:{random.choice(system)}\n\n')
    instructions.append('\n'.join(results))
    
len(instructions)

100%|█████████████████████████████████| 51876/51876 [00:00<00:00, 110426.28it/s]


52144

In [9]:
instructions[0]

'<manusia>:u always reply in tamil\n\n நீங்கள் ஒரு ஆழமான தொழில்நுட்ப தொழில்முனைவோர் உங்கள் அடுத்த உலகத்தை மாற்றும் தொடக்க தேடும். நீங்கள் இயற்பியல் மற்றும் பொறியியல் ஒரு பின்னணி உள்ளது, மற்றும் நீங்கள் குறிப்பாக ஆற்றல் ஆர்வமாக உள்ளன, காலநிலை, விண்வெளி, பொருட்கள், மற்றும் அரைக்கடத்திகள், ஆனால் நீங்கள் எந்த ஆழமான தொழில்நுட்ப துறையில் திறந்த உள்ளன. அடுத்ததாக வேலை செய்ய 15 தொடக்க யோசனைகளை பரிந்துரைக்கவும். \n<bot>: 1. மேம்பட்ட சோலார் பேனல் உற்பத்தி: பொருள் அறிவியல் மற்றும் பொறியியலில் முன்னேற்றங்களை மேம்படுத்துவதன் மூலம் மிகவும் திறமையான மற்றும் மலிவு விலையில் புதிய தலைமுறை சோலார் பேனல்களை உருவாக்குங்கள். 2. ஃப்யூஷன் எரிசக்தி: நடைமுறை ஃப்யூஷன் எரிசக்தி தீர்வுகளை உருவாக்குவதில் கவனம் செலுத்தும் ஒரு தொடக்கத்தை உருவாக்குதல், உலகளாவிய எரிசக்தி நிலப்பரப்பை மாற்றியமைத்தல் மற்றும் காலநிலை மாற்றத்தை கையாள்வது. 3. கார்பன் பிடிப்பு மற்றும் பயன்பாடு: கார்பன் டை ஆக்சைடு உமிழ்வுகளை எரிபொருள்கள், இரசாயனங்கள் அல்லது கட்டுமானப் பொருட்கள் போன்ற மதிப்புமிக்க தயாரிப்புகளாகப் பிடித்து மாற்றுவதற்கான புதுமையான 

In [10]:
filtered = []
for f in instructions:
    filtered.append({
        'prompt_input': None,
        'input': f,
        'output': None,
    })

In [11]:
import pandas as pd

pd.DataFrame(filtered).to_parquet('prepared-force-tamil.parquet')

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-force-tamil.parquet",
    path_in_repo='data/force_tamil-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-force-tamil.parquet:   0%|          | 0.00/40.7M [00:00<?, ?B/s]